<a href="https://colab.research.google.com/github/hsnam95/class2023Spring/blob/main/whisper_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Whisper demo** 
Whisper is a general-purpose speech recognition model. It is trained on a large dataset of diverse audio and is also a multi-task model that can perform multilingual speech recognition as well as speech translation and language identification. This demo cuts audio after around 30 secs.



### 1. Install dependencies

In [ ]:
!git clone https://huggingface.co/spaces/openai/whisper
%cd whisper
!pip install -r requirements.txt
!pip install gradio

fatal: destination path 'whisper' already exists and is not an empty directory.
/content/whisper
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### 2. Run

Run this cell to show the gradio app

In [ ]:
import os
os.system("pip install git+https://github.com/openai/whisper.git")
import gradio as gr
import whisper

from share_btn import community_icon_html, loading_icon_html, share_js
model = whisper.load_model("large-v2")

100%|█████████████████████████████████████| 2.87G/2.87G [00:55<00:00, 55.5MiB/s]


In [ ]:
def inference(audio):
    audio = whisper.load_audio(audio)
    audio = whisper.pad_or_trim(audio)
    
    mel = whisper.log_mel_spectrogram(audio).to(model.device)
    
    _, probs = model.detect_language(mel)
    
    options = whisper.DecodingOptions(fp16 = False)
    result = whisper.decode(model, mel, options)
    return result.text

block = gr.Blocks()

with block:
    audio = gr.Audio(
                source="microphone",
                type="filepath"
            )
    btn = gr.Button("Transcribe")
    text = gr.Textbox()
    btn.click(inference, inputs=audio, outputs=text)
block.launch()

In [5]:
!pip install transformers
import torch
from transformers import pipeline
# !pip install datasets
# from datasets import load_dataset


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
# TPU로 설정하면 comment out the below
device = "cuda:0" if torch.cuda.is_available() else "cpu" 
pipe = pipeline(
  "automatic-speech-recognition",
  model="openai/whisper-large-v2",
  chunk_length_s=30,
  device=device,
)

# ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
# sample = ds[0]["audio"]

# prediction = pipe(sample.copy(), batch_size=8)["chunks"]

# # we can also return timestamps for the predictions
# prediction = pipe(sample.copy(), batch_size=8, return_timestamps=True)["chunks"]


In [14]:
!pip install gradio
import gradio as gr

def inference(audio):
  # stride_length_s=[6,0], as input below
  prediction = pipe(audio, batch_size=5, return_timestamps=True, generate_kwargs = {"language":"<|ko|>","task":"transcribe"})["chunks"]

  return prediction

block = gr.Blocks()
with block:
    audio = gr.Audio(
                source="microphone",
                type="filepath"
            )
    btn = gr.Button("Transcribe")
    text = gr.Textbox()
    btn.click(inference, inputs=audio, outputs=text)
block.launch()



# gr.Interface(
#     fn=inference, 
#     inputs=gr.Audio(source="microphone", type="filepath"), 
#     outputs="text").launch()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>